## Data

In [20]:
import pandas as pd

Using the cleaned data

In [21]:
df = pd.read_csv('matched_texts.csv') 

In [22]:
abusive_df = df[(df['abusetag2'] == 1)]
nonabusive_df = df[(df['abusetag2'] == 0)]

In [23]:
sampled_abusive_df = abusive_df.sample(n=100, random_state=42).sample(frac=1, random_state=42).reset_index(drop=True)
sampled_nonabusive_df = nonabusive_df.sample(n=100, random_state=42).sample(frac=1, random_state=42).reset_index(drop=True)
sampled_df = pd.concat([sampled_nonabusive_df, sampled_abusive_df])
sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [24]:
sampled_df

,id,abusetag2,tigger.predict,tigger.predprob,tiggerpred_0,tiggerpred_1,text
0,1.625890e+18,0,0,0.910161,0.910161,0.089839,@LCP @olivierdussopt @Elisabeth_Borne Par cont...
1,1.623221e+18,0,0,0.917249,0.917249,0.082751,@shatanou @LaFrancematuer @sandrousseau @EELV ...
2,1.625263e+18,0,0,0.952877,0.952877,0.047123,@rvicot @faureolivier @BorisVallaud @MartineAu...
3,1.630572e+18,1,1,0.942305,0.057695,0.942305,@Deputee_Obono Mange tes morts
4,1.626732e+18,1,1,0.751286,0.248714,0.751286,RT @karineS34140: @LouisBoyard c’est deux sema...
...,...,...,...,...,...,...,...
195,1.624355e+18,1,1,0.990829,0.009171,0.990829,"@lamiaela @Portes_Thomas Ferme la, idiote!"
196,1.624898e+18,0,0,0.900513,0.900513,0.099487,شبينغلر https://twitter.com/Ahmedsa1357913/sta...
197,1.626490e+18,0,0,0.994710,0.994710,0.005290,@Fabrice_TeamEM @Ugobernalicis @YaelBRAUNPIVET...
198,1.622701e+18,1,1,0.664141,0.335859,0.664141,@sandrousseau @Clem_Autain 🤦‍♀️🤦‍♀️🤦‍♀️Seigneu...


## Ask LLM

In [25]:
texts_to_analyze = sampled_df['text'].tolist()

In [26]:
from openai import OpenAI
import json
import re

client = OpenAI(
    api_key = "abc", # Change this to your OpenAI API key
)

In [27]:
def get_abusive_score(client, text_to_analyse, file_name):
    tweets_content = "\n".join([f"{i+1}. {tweet}" for i, tweet in enumerate(text_to_analyse)])
    
    messages = [
        {"role": "system", "content": "You are an annotator for the tweets in French."},
        {"role": "user", "content": (
            "Now you are an annotator for the tweets which are the responses for French politicians. For each tweet in the given list, annotate the given tweets as abusive or not abusive in the format of : "
            "{'index':'score'} which can be saved into a json file directly (separating with ',', without any '/n'). For the score, abusive tweets should be labelled as 1, non-abusive tweets should be labelled as 0.\n\n"
            "When annotating the tweets, please consider the following coding scheme:\n"
            "Abusive: Negative, toxic, aggressive, threatening and demeaning comments that targets the MP as an individual (and not as a politician). These include:\n"
            "1) slurs and hate speech (including censored terms): Direct and explicit words that are commonly classified as hateful towards an individual or group based on identity characteristics. "
            "This also includes instances of the word being included in its censored version to get around automatic content moderation (for example p*** = “pute”)\n"
            "2) aggressive or demeaning language: Insulting, aggressive, toxic, demeaning and belittling language targeting the MP as an individual (but not necessarily crossing the line to illegal hate speech).\n"
            "3) Personal negative remarks: Any comment on personal characteristics that do not have anything to do with the person’s duties as politicians. These include comments on dress, personal appearance, voice, and other personal aspects not related to their duties as politician. Note that these comments can sometimes seem positive at face value (“tu es belle”) but be considered abusive in context as they aim to reduce the MP to their physical appearance.\n"
            "4) Threats: Evoking threats of physical or sexual violence (directly or indirectly).\n"
            "Attention, everything that is negative but explicitly targets the individual's behavior as a politician and their policies, rather than the individual as a person, should be labelled as 'non-abusive'. Pay attention to suspected cases of sarcasm: for example 'waouh il est intelligent........' is different from 'comme il est intelligent !👏'"
            "Here are the tweets:\n" + tweets_content)}
    ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        temperature=0.5,
    )
    
    response_message = response.choices[0].message.content

    response_message_clean = re.sub(r'[\n\t]', '', response_message)
    
    print(response_message_clean)
    
    try:
        annotations = json.loads(response_message_clean)
    except json.JSONDecodeError as e:
        print(f"JSON decoding error: {e}")
        return
    
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(annotations, f, ensure_ascii=False, indent=4)

In [28]:
get_abusive_score(client, texts_to_analyze, 'updated_1_emoji_data_gpt_3_5.json')

{"1": "0","2": "0","3": "0","4": "1","5": "1","6": "1","7": "1","8": "1","9": "1","10": "0","11": "1","12": "1","13": "0","14": "0","15": "0","16": "1","17": "1","18": "1","19": "1","20": "1","21": "1","22": "0","23": "1","24": "1","25": "0","26": "0","27": "0","28": "0","29": "1","30": "1","31": "0","32": "0","33": "0","34": "0","35": "1","36": "1","37": "1","38": "1","39": "1","40": "0","41": "1","42": "1","43": "1","44": "0","45": "1","46": "1","47": "1","48": "1","49": "0","50": "0","51": "1","52": "0","53": "1","54": "1","55": "1","56": "1","57": "1","58": "0","59": "1","60": "1","61": "1","62": "1","63": "1","64": "0","65": "1","66": "0","67": "1","68": "1","69": "1","70": "1","71": "0","72": "1","73": "1","74": "1","75": "1","76": "1","77": "1","78": "1","79": "1","80": "1","81": "1","82": "0","83": "1","84": "1","85": "1","86": "1","87": "1","88": "1","89": "1","90": "1","91": "1","92": "1","93": "1","94": "1","95": "1","96": "1","97": "1","98": "1","99": "1","100": "1","101": 

In [31]:
def get_abusive_score(client, text_to_analyse, file_name):
    tweets_content = "\n".join([f"{i+1}. {tweet}" for i, tweet in enumerate(text_to_analyse)])
    
    messages = [
        {"role": "system", "content": "You are an annotator for tweets in French."},
        {"role": "user", "content": (
            '''
            Now you are an annotator for tweets targeting French politicians. For each tweet in the given list, annotate the given tweets as abusive or not abusive in the format of : "
            {'index':'score'}, for instance {"1": 0,"2": 0,"3": 0,...}, do not give me the json file directly. For the score, abusive tweets should be labelled as 1, non-abusive tweets should be labelled as 0.\n\n"
            When annotating the tweets, please consider the following coding scheme:\n"
            Abusive: Negative, toxic, aggressive, threatening and demeaning comments that targets the MP as an individual (and not as a politician). These include:\n"
            1) slurs and hate speech (including censored terms): Direct and explicit words that are commonly classified as hateful towards an individual or group based on identity characteristics. "
            This also includes instances of the word being included in its censored version to get around automatic content moderation (for example p*** = “pute”)\n"
            2) aggressive or demeaning language: Insulting, aggressive, toxic, demeaning and belittling language targeting the MP as an individual (but not necessarily crossing the line to illegal hate speech).\n"
            3) Personal negative remarks: Any comment on personal characteristics that do not have anything to do with the person’s duties as politicians. These include comments on dress, personal appearance, voice, and other personal aspects not related to their duties as politician. Note that these comments can sometimes seem positive at face value (“tu es belle”) but be considered abusive in context as they aim to reduce the MP to their physical appearance.\n"
            4) Threats: Evoking threats of physical or sexual violence (directly or indirectly).\n\n
            Attention, Everything that is negative but explicitly targets the individual's behavior as a politician and their policies, rather than the individual as a person, should be labelled as 'non-abusive'. Pay attention to suspected cases of sarcasm: for example 'waouh il est intelligent........' is different from 'comme il est intelligent !👏'
            '''
            "Here are the tweets:\n" + tweets_content)}
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.5,
    )
    
    response_message = response.choices[0].message.content

    response_message_clean = re.sub(r'[\n\t]', '', response_message)
    
    print(response_message_clean)
    
    
    try:
        annotations = json.loads(response_message_clean)
    except json.JSONDecodeError as e:
        print(f"JSON decoding error: {e}")
        return
    
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(annotations, f, ensure_ascii=False, indent=4)

In [32]:
get_abusive_score(client, texts_to_analyze, 'updated_1_emoji_data_gpt_4.json')

{"1": 0, "2": 0, "3": 0, "4": 1, "5": 1, "6": 1, "7": 1, "8": 1, "9": 1, "10": 0, "11": 0, "12": 1, "13": 0, "14": 0, "15": 0, "16": 1, "17": 0, "18": 1, "19": 0, "20": 1, "21": 1, "22": 0, "23": 1, "24": 0, "25": 0, "26": 0, "27": 0, "28": 0, "29": 1, "30": 0, "31": 0, "32": 0, "33": 0, "34": 0, "35": 1, "36": 1, "37": 0, "38": 1, "39": 1, "40": 0, "41": 0, "42": 1, "43": 0, "44": 0, "45": 1, "46": 1, "47": 0, "48": 0, "49": 1, "50": 0, "51": 1, "52": 0, "53": 0, "54": 1, "55": 1, "56": 1, "57": 1, "58": 0, "59": 1, "60": 0, "61": 1, "62": 0, "63": 0, "64": 0, "65": 0, "66": 0, "67": 1, "68": 0, "69": 1, "70": 1, "71": 0, "72": 0, "73": 1, "74": 0, "75": 1, "76": 1, "77": 0, "78": 0, "79": 1, "80": 0, "81": 0, "82": 0, "83": 0, "84": 1, "85": 0, "86": 0, "87": 1, "88": 0, "89": 1, "90": 0, "91": 0, "92": 0, "93": 0, "94": 0, "95": 0, "96": 1, "97": 0, "98": 0, "99": 0, "100": 1, "101": 0, "102": 0, "103": 1, "104": 0, "105": 0, "106": 1, "107": 0, "108": 0, "109": 1, "110": 0, "111": 

## Get the corresponding text

In [42]:
with open('updated_1_emoji_data_gpt_3_5.json', 'r') as file:
    annotations_gpt_3_5 = json.load(file)

In [43]:
with open('updated_1_emoji_data_gpt_4.json', 'r') as file:
    annotations_gpt_4 = json.load(file)

In [44]:
df_gpt_4 = pd.DataFrame(annotations_gpt_4, index=[0])
df_gpt_3_5 = pd.DataFrame(annotations_gpt_3_5, index=[0])

In [45]:
df_gpt_4 = df_gpt_4.T
df_gpt_3_5 = df_gpt_3_5.T

In [46]:
df_gpt_4 = df_gpt_4.rename(columns={df_gpt_4.columns[0]: 'gpt4_score'}).reset_index(drop=True)
df_gpt_3_5 = df_gpt_3_5.rename(columns={df_gpt_3_5.columns[0]: 'gpt3_5_score'}).reset_index(drop=True)

In [47]:
merged_df = sampled_df.merge(df_gpt_4, left_index=True, right_index=True)
merged_df = merged_df.merge(df_gpt_3_5, left_index=True, right_index=True)

In [48]:
final_df = merged_df[['text', 'abusetag2', 'gpt3_5_score', 'gpt4_score']]

In [49]:
final_df
final_df.to_csv('updated_1_emoji_data_LLM_score.csv', index=False)